In [1]:
import logging

from configuration.llm_inference_configuration import APILLMConfiguration
from configuration.embedding_inference_configuration import APIEmbeddingModelConfiguration, LocalEmbeddingModelConfiguration
from llm.azure_openai import AzureOpenAILLM
from embedding.local_embedding import LocalEmbeddingModel
from embedding.request_embedding import RequestEmbeddingModel
from prompt.zero_shot import ZeroShotPrompt
from prompt.few_shot import FewShotPrompt

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s\t\t%(levelname)s\t%(message)s',
)

c:\Users\ADMIN\miniconda3\Lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


# 1. System component test

## 1.1. LLM test

### 1.1.1. Azure OpenAI test

In [2]:
gpt_4o_azure_configuration = APILLMConfiguration()
gpt_4o_azure_configuration.load("configuration/yaml/llm/azure_gpt_4o.yaml")
azure_gpt_4o = AzureOpenAILLM(gpt_4o_azure_configuration)

try:
    azure_gpt_4o_1 = AzureOpenAILLM(gpt_4o_azure_configuration)
except Exception as e:
    logging.info("LLM unique ID test successful")
    logging.error(f"Failed to create AzureOpenAILLM instance: {e}")

2025-05-03 15:04:08,975		DEBUG	[🔧 CONFIGURATION] List of APILLMConfiguration's properties: dict_keys(['llm_id', 'model.model_name', 'model.temperature', 'model.max_tokens', 'cache.enabled', 'cache.cache_expiry', 'llm_api.api_key', 'llm_api.api_base', 'llm_api.api_version', 'llm_api.deployment_name', 'cost.prompt_token_cost', 'cost.response_token_cost', 'retry.max_retries', 'retry.backoff_factor', 'security.encrypted', 'security.trust_strategy'])
2025-05-03 15:04:08,976		DEBUG	[🔧 CONFIGURATION] List of APILLMConfiguration's sensitive properties: []
2025-05-03 15:04:08,980		DEBUG	🔧 System Component ID: SYSTEM_COMPONENT | 0 | Component type: <class 'llm.azure_openai.AzureOpenAILLM'>
2025-05-03 15:04:08,980		INFO	✅ LLM config loaded: azure_gpt_4o
2025-05-03 15:04:08,985		DEBUG	Starting new HTTPS connection (1): vtnet-ai-service-swedencentral.cognitiveservices.azure.com:443
2025-05-03 15:04:10,681		DEBUG	https://vtnet-ai-service-swedencentral.cognitiveservices.azure.com:443 "POST //openai/d

In [3]:
sample_zero_shot_message = ZeroShotPrompt(
    [
        {
            "role": "user", 
            "content": "Health check. Say 'hi' to start the conversation."
        }
    ]
)

sample_zero_shot_message_responses = azure_gpt_4o.query(query=sample_zero_shot_message.prompt, num_responses=2)
print(azure_gpt_4o.get_response_texts(sample_zero_shot_message_responses))

2025-04-29 16:06:27,592		DEBUG	Request options: {'method': 'post', 'url': '/deployments/gpt-4o/chat/completions', 'headers': {'api-key': '<redacted>'}, 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "Health check. Say 'hi' to start the conversation."}], 'model': 'gpt-4o', 'max_tokens': 2048, 'n': 2, 'stop': ['\n'], 'temperature': 0.2}}
2025-04-29 16:06:27,594		DEBUG	Sending HTTP Request: POST https://vtnet-ai-service-swedencentral.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview
2025-04-29 16:06:27,595		DEBUG	connect_tcp.started host='vtnet-ai-service-swedencentral.cognitiveservices.azure.com' port=443 local_address=None timeout=5.0 socket_options=None


2025-04-29 16:06:27,862		DEBUG	connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000021938A6EDD0>
2025-04-29 16:06:27,863		DEBUG	start_tls.started ssl_context=<ssl.SSLContext object at 0x0000021936125370> server_hostname='vtnet-ai-service-swedencentral.cognitiveservices.azure.com' timeout=5.0
2025-04-29 16:06:28,302		DEBUG	start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000021938A7BC90>
2025-04-29 16:06:28,303		DEBUG	send_request_headers.started request=<Request [b'POST']>
2025-04-29 16:06:28,304		DEBUG	send_request_headers.complete
2025-04-29 16:06:28,304		DEBUG	send_request_body.started request=<Request [b'POST']>
2025-04-29 16:06:28,306		DEBUG	send_request_body.complete
2025-04-29 16:06:28,306		DEBUG	receive_response_headers.started request=<Request [b'POST']>
2025-04-29 16:06:28,817		DEBUG	receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Length', b'1745'), (b'Content-Type', b'appli

['Hi! How can I assist you with your health check today?', 'Hi! How can I assist you with your health check today?']


In [4]:
sample_few_shots_message = FewShotPrompt(
    [
        {
            "role": "system", 
            "content": "You are NetMind assistant. You task is to answer to the user anything about Viettel Group."
        },
        {
            "role": "user", 
            "content": "Tell me about Viettel Group."
        },
        {
            "role": "assistant", 
            "content": "Viettel Group is a Vietnamese multinational telecommunications company headquartered in Hanoi, Vietnam. It is a state-owned enterprise and operated by the Ministry of Defence. You can find out more about Viettel Group at https://viettel.vn/."
        },
        {
            "role": "user",
            "content": "What is the revenue of Viettel Group?"
        }
    ]
)

sample_few_shots_message_responses = azure_gpt_4o.query(query=sample_few_shots_message.prompt, num_responses=2)
print(azure_gpt_4o.get_response_texts(sample_few_shots_message_responses))

2025-04-29 16:07:02,379		DEBUG	Request options: {'method': 'post', 'url': '/deployments/gpt-4o/chat/completions', 'headers': {'api-key': '<redacted>'}, 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'You are NetMind assistant. You task is to answer to the user anything about Viettel Group.'}, {'role': 'user', 'content': 'Tell me about Viettel Group.'}, {'role': 'assistant', 'content': 'Viettel Group is a Vietnamese multinational telecommunications company headquartered in Hanoi, Vietnam. It is a state-owned enterprise and operated by the Ministry of Defence. You can find out more about Viettel Group at https://viettel.vn/.'}, {'role': 'user', 'content': 'What is the revenue of Viettel Group?'}], 'model': 'gpt-4o', 'max_tokens': 2048, 'n': 2, 'stop': ['\n'], 'temperature': 0.2}}
2025-04-29 16:07:02,380		DEBUG	Sending HTTP Request: POST https://vtnet-ai-service-swedencentral.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-0

["As of the latest available data, Viettel Group reported a revenue of approximately 274 trillion Vietnamese Dong (VND) in 2022. However, please note that financial figures can vary year by year, and it's advisable to check the most recent reports or official announcements for the latest figures.", "As of the latest available data, Viettel Group reported a revenue of approximately 274 trillion VND (Vietnamese Dong) in 2022, which is roughly equivalent to around 11.8 billion USD. This figure reflects Viettel's strong position in the telecommunications industry, both domestically and internationally. Keep in mind that financial figures can vary year by year, so it's always a good idea to check the most recent reports for updated information."]


## 1.2. Embedding test

In [2]:
all_mini_v2_configuration = APIEmbeddingModelConfiguration()
all_mini_v2_configuration.load("configuration/yaml/embedding/all-MiniLM-L6-v2.yaml")
bge_m3_hf = RequestEmbeddingModel(all_mini_v2_configuration)

2025-05-03 15:40:23,628		DEBUG	[🔧 APIEmbeddingModelConfiguration] List of APIEmbeddingModelConfiguration's properties: dict_keys(['emb_id', 'model.model_name', 'model.max_tokens', 'model.embedding_dims', 'model.identical_threshold', 'emb_api.api_base', 'emb_api.api_token', 'emb_api.trust_remote_code', 'cost.prompt_token_cost', 'cost.response_token_cost', 'retry.max_retries', 'retry.backoff_factor'])
2025-05-03 15:40:23,628		DEBUG	[🔧 APIEmbeddingModelConfiguration] List of APIEmbeddingModelConfiguration's sensitive properties: []
2025-05-03 15:40:23,643		DEBUG	[🔧 RequestEmbeddingModel] System Component ID: SYSTEM_COMPONENT | 0 | Component type: <class 'embedding.request_embedding.RequestEmbeddingModel'>
2025-05-03 15:40:23,644		INFO	[✅ RequestEmbeddingModel] Embedding config loaded: all-MiniLM-L6-v2
2025-05-03 15:40:23,644		DEBUG	List of APIEmbeddingModelConfiguration: {'_properties': {'emb_id': {'default-value': '', 'transform_fn': <class 'str'>}, 'model.model_name': {'default-value': 

ValueError: [❌ RequestEmbeddingModel] Failed to connect to API embedding model: 404, {"error":"Model all-MiniLM-L6-v2 does not exist"}

In [3]:
bge_m3_hf.encode("Hello world")

2025-05-03 15:16:01,399		DEBUG	Starting new HTTPS connection (1): api-inference.huggingface.co:443
2025-05-03 15:16:03,409		DEBUG	https://api-inference.huggingface.co:443 "POST /models/BAAI/bge-m3 HTTP/1.1" 503 None
2025-05-03 15:16:03,412		WARNING	[❌ RequestEmbeddingModel] HTTP error occurred: 503 Server Error: Service Temporarily Unavailable for url: https://api-inference.huggingface.co/models/BAAI/bge-m3. Retrying 1/5...
2025-05-03 15:16:05,416		DEBUG	Starting new HTTPS connection (1): api-inference.huggingface.co:443
2025-05-03 15:16:06,253		DEBUG	https://api-inference.huggingface.co:443 "POST /models/BAAI/bge-m3 HTTP/1.1" 503 None
2025-05-03 15:16:06,255		WARNING	[❌ RequestEmbeddingModel] HTTP error occurred: 503 Server Error: Service Temporarily Unavailable for url: https://api-inference.huggingface.co/models/BAAI/bge-m3. Retrying 2/5...
2025-05-03 15:16:10,258		DEBUG	Starting new HTTPS connection (1): api-inference.huggingface.co:443
2025-05-03 15:16:12,228		DEBUG	https://api-in

ValueError: [❌ RequestEmbeddingModel] Max retries exceeded. Failed to get embedding.

In [4]:
bge_m3_hf = RequestEmbeddingModel(bge_m3_configuration)

2025-05-03 15:06:10,099		DEBUG	🔧 System Component ID: SYSTEM_COMPONENT | 0 | Component type: <class 'embedding.request_embedding.RequestEmbeddingModel'>
2025-05-03 15:06:10,101		INFO	✅ Embedding config loaded: bge-m3
2025-05-03 15:06:10,101		DEBUG	List of APIEmbeddingModelConfiguration: {'_properties': {'emb_id': {'default-value': '', 'transform_fn': <class 'str'>}, 'model.model_name': {'default-value': '', 'transform_fn': <class 'str'>}, 'model.max_tokens': {'default-value': 0, 'transform_fn': <class 'int'>}, 'model.embedding_dims': {'default-value': 0, 'transform_fn': <class 'int'>}, 'model.identical_threshold': {'default-value': 0.999, 'transform_fn': <class 'float'>}, 'emb_api.api_base': {'default-value': '', 'transform_fn': <class 'str'>}, 'emb_api.api_token': {'default-value': '', 'transform_fn': <class 'str'>}, 'emb_api.trust_remote_code': {'default-value': False, 'transform_fn': <class 'bool'>}, 'cost.prompt_token_cost': {'default-value': 0.0, 'transform_fn': <class 'float'>}, 

AttributeError: 'APIEmbeddingModelConfiguration' object has no attribute 'emb_api_api_token'

In [7]:
a = {"b": 1, "c": 2}
list(a.keys())

['b', 'c']

## 2. Test tools

### 2.1. Python code runner tool

In [ ]:
from tools.python_code_runner import PythonCodeRunnerTool
from configuration.tool_configuration import ToolConfiguration

In [ ]:
python_code_runner_config = ToolConfiguration()
python_code_runner_config.load("configuration/yaml/tools/python_code_runner.yaml")
python_code_runner = PythonCodeRunnerTool(python_code_runner_config)

In [ ]:
input_code_list = [
    {
        "code_string": "a = 11\nb = 20\nresult = a + b\n\ndef greet(name):\n return f'Hello, {name}!'\n \nmessage = greet('World')",
        "id": "1"
    },
    {
        "code_string": "def multiply(x, y):\n\treturn x * y\n\nnum1 = 5\nnum2 = 3\nresult = multiply(num1, num2)\n\nprint(f'The product of {num1} and {num2} is {result}.')",
        "id": "2"
    }
]

results = python_code_runner.execute(input_code_list=input_code_list)
results

## 3. Test memory

In [ ]:
from configuration.llm_inference_configuration import APILLMConfiguration
from llm.azure_openai import AzureOpenAILLM
from base_classes.memory.memory_atom import AbstractMemoryAtom
from base_classes.memory.memory_block import AbstractMemoryBlock
from base_classes.memory.memory import AbstractMemory
from base_classes.memory.datatypes.data_item import PromptDataItem
from prompt.user_message import UserMessagePrompt
from prompt.assistant_message import AssistantMessagePrompt
from base_classes.memory.memory_feature_engineer import MemoryFeatureEngineer

In [2]:
prompt_data_1 = [{
    'role': 'user',
    'content': 'Calculate the sum of 11 and 20.'
}]
mem_atom_1 = AbstractMemoryAtom(data=PromptDataItem(UserMessagePrompt(prompt_data_1)))

prompt_data_2 = [{
    'role': 'assistant',
    'content': 'In order to calculate the sum of two numbers, we need to do the following:\n\t1. Write a Python code with add_sum() function, receiving 2 variables.\n\t2. Execute the code with the given variables.\n\t3. Return the result of the sum.'
}]
mem_atom_2 = AbstractMemoryAtom(data=PromptDataItem(AssistantMessagePrompt(prompt_data_2)))

prompt_data_3 = [{
    'role': 'assistant',
    'content': 'Here is the Python code to calculate the sum of two numbers:\n\na = 11\nb = 20\nresult = a + b\n\nprint(f"The sum of {a} and {b} is {result}.")'
}]
mem_atom_3 = AbstractMemoryAtom(data=PromptDataItem(AssistantMessagePrompt(prompt_data_3)))

prompt_data_4 = [{
    'role': 'assistant',
    'content': 'After executing the code, we got the sum of 11 and 20 is 31.'
}]
mem_atom_4 = AbstractMemoryAtom(data=PromptDataItem(AssistantMessagePrompt(prompt_data_4)))

In [3]:
AbstractMemoryAtom.get_mematom_ids()

[UUID('502d1013-044e-46c0-b3e1-755f5e4cfd1c'),
 UUID('a13de8b8-ec81-4af4-a6bf-33f6e7c93b33'),
 UUID('ed6b3e6e-bcd4-4a97-8566-7fd562983cd3'),
 UUID('18fd9901-2553-43fe-bbf6-f9ce14418a89')]

In [4]:
mem_block = AbstractMemoryBlock()
mem_block.add_memory_atom(mem_atom_1)
mem_block.add_memory_atom(mem_atom_2)
mem_block.add_memory_atom(mem_atom_3)
mem_block.add_memory_atom(mem_atom_4)

mem_block.mem_atom_graph = {
        mem_atom_1.mem_atom_id: [mem_atom_2.mem_atom_id],
        mem_atom_2.mem_atom_id: [mem_atom_3.mem_atom_id, mem_atom_4.mem_atom_id],
        mem_atom_3.mem_atom_id: [],
        mem_atom_4.mem_atom_id: []
    }

In [6]:
memory = AbstractMemory()
memory.add_memory_block(mem_block)

AttributeError: 'AbstractMemory' object has no attribute '_memory_fe'

In [ ]:
memory.get_memory_block_by_id(mem_block.mem_block_id)

In [17]:
from annotated_docs.json_schema import as_json_schema

class A:
    """
    This is a class A.
    """
    
    def __init__(self, a: int, b: str):
        """
        This is the constructor of class A.
        """
        self.a = a
        self.b = b
    @classmethod
    def get_a(self, m: int) -> int:
        """
        This is the get_a method of class A.
        """
        return self.a
    
    def get_b(self) -> str:
        """
        This is the get_b method of class A.
        """
        return self.b

a = A(a = 2, b = "a")

as_json_schema(A.get_a)
# a = A(a = 2, b = "a")
# as_json_schema(A.get_a())

{'name': 'get_a',
 'description': 'This is the get_a method of class A.',
 'parameters': {'properties': {'m': {'type': 'integer'}},
  'required': ['m'],
  'type': 'object'}}